Notebook to play around with pulling the library data and displaying it on a map plus pulling in some census data.

In [ ]:
%matplotlib inline
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import ottawa_libraries as ol

### Pull library data

In [ ]:
ottawa_locations = ol.get_ottawa_locations()

### Open and explore map data

In [ ]:
# open and plot the data. looks like Toronto
census_file = 'data/census_tracts.shp'
canada_census = gpd.read_file(census_file)

In [ ]:
canada_census.CMANAME.unique()

### Get and explore Ottawa map data

In [ ]:
ottawa_census = canada_census[(canada_census.CMANAME=="Ottawa - Gatineau (Ontario part / partie de l'Ontario)")]
ottawa_census.plot()

In [ ]:
# Changing size
fig, ax = plt.subplots(figsize=(20,20))
ottawa_census.plot(ax=ax)

### Convert library data to pandas

In [ ]:
libraries = pd.DataFrame(ottawa_locations)
# pivot the dataframe
libraries = libraries.T.reset_index()
libraries = libraries.rename(columns={'index':'Name'})

In [ ]:
libraries.head()

In [ ]:
libraries['Latitude'] = pd.to_numeric(libraries['Latitude'])
libraries['Longitude'] = pd.to_numeric(libraries['Longitude'])

### Convert library data to geopandas

In [ ]:
libraries['geometry'] = libraries.apply(lambda x:
                                                shapely.geometry.point.Point(x['Longitude'], x['Latitude']),
                                                axis=1)
libraries_gpd = gpd.GeoDataFrame(libraries)

In [ ]:
libraries_gpd.head()

### Plot Ottawa and library data

In [ ]:
# Let's plot the libraries onto the map
fig, ax = plt.subplots(figsize=(20,20))
ottawa_census.plot(ax=ax)
libraries_gpd.plot(ax=ax, markersize=10, color='red')

In [ ]:
ottawa_census = ottawa_census.to_crs('epsg:4326')

In [ ]:
# Let's plot the libraries onto the map
fig, ax = plt.subplots(figsize=(20,20))
ottawa_census.plot(ax=ax)
libraries_gpd.plot(ax=ax, markersize=10, color='red')

### Open and explore census data

In [ ]:
pop_data = pd.read_csv('data/98-401-X2016043_eng_CSV/98-401-X2016043_English_CSV_data.csv')

In [ ]:
pop_data.head()

### Restrict census data

In [ ]:
len(ottawa_census.CTUID)

In [ ]:
len(pop_data[pop_data['GEO_CODE (POR)'].isin(ottawa_census.CTUID)])

In [ ]:
len(pop_data[pop_data['GEO_CODE (POR)'].isin(ottawa_census.CTUID)]['GEO_CODE (POR)'].unique())

In [ ]:
ottawa_pop_data = pop_data[pop_data['GEO_CODE (POR)'].isin(ottawa_census.CTUID)]

In [ ]:
len(ottawa_pop_data)

In [ ]:
len(ottawa_pop_data['GEO_CODE (POR)'].unique())

In [ ]:
len(ottawa_pop_data.loc[ottawa_pop_data["DIM: Profile of Census Tracts (2247)"]=='Population, 2016'])

In [ ]:
ottawa_pop_data = ottawa_pop_data.loc[ottawa_pop_data["DIM: Profile of Census Tracts (2247)"]=='Population, 2016']

In [ ]:
ottawa_pop_data['pop'] = ottawa_pop_data['Dim: Sex (3): Member ID: [1]: Total - Sex']

In [ ]:
ottawa_pop_data['pop'] = pd.to_numeric(ottawa_pop_data['pop'])

Below doesn't quite line up with the data [here](https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=CSD&Geo2=PR&Code2=01&Data=Count&SearchType=Begins&SearchPR=01&B1=All&Code1=3506008) where it shows 934,243. General ballpark. Of note, the census tracts shown here are for `Ottawa - Gatineau (Ontario part / partie de l'Ontario)` so I hypothesise that we're getting a couple tracts beyond just Ottawa. __Should come back and validate!__

In [ ]:
ottawa_pop_data['pop'].sum()

### Join map data and census data

In [ ]:
ottawa_census_pd = pd.DataFrame(ottawa_census)
ottawa_census_pd['CTUID'] = pd.to_numeric(ottawa_census_pd['CTUID'])
ottawa_census_pd = ottawa_census_pd.merge(right=ottawa_pop_data, how='left',
                              left_on='CTUID', right_on='GEO_CODE (POR)')

In [ ]:
ottawa_census_gpd = gpd.GeoDataFrame(ottawa_census_pd)

### Plot with population as colour

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ottawa_census_gpd.plot(ax=ax, column='pop', cmap='Blues')
libraries_gpd.plot(ax=ax, markersize=10, color='red')

In [ ]:
ottawa_census_gpd = ottawa_census_gpd.to_crs('epsg:4326')

### Calculate and plot population density

In [ ]:
"""
keep getting error:
UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect.
Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.
Unclear why given I'm doing this in the cell above.
"""
ottawa_census_gpd['density'] = ottawa_census_gpd['pop']/ottawa_census_gpd.area

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ottawa_census_gpd.plot(ax=ax, column='density', cmap='Blues')
libraries_gpd.plot(ax=ax, markersize=10, color='red')

In [ ]:
ottawa_census_gpd.crs

### Check out interactive map

In [ ]:
ottawa_census_gpd['area'] = ottawa_census_gpd.area
ottawa_census_display = ottawa_census_gpd[['geometry', 'pop', 'density', 'area']]

In [ ]:
m = ottawa_census_display.explore(column='density')
libraries_gpd.explore(m=m, color='red')

### Look at density distributions
Was really shocked how few tracts have really high denisty

In [ ]:
ottawa_census_gpd.density.plot(kind='hist', bins=15, figsize=(13,5))

### Pulling in area from census data
Should ensure accuracy and avoid issues like including bits of river/lake.

In [ ]:
ottawa_area_data = pop_data[pop_data['GEO_CODE (POR)'].isin(ottawa_census.CTUID)]

In [ ]:
ottawa_area_data = ottawa_area_data.loc[ottawa_area_data["DIM: Profile of Census Tracts (2247)"]=='Land area in square kilometres']

In [ ]:
ottawa_area_data = ottawa_area_data[['GEO_CODE (POR)', 'Dim: Sex (3): Member ID: [1]: Total - Sex']]
ottawa_area_data=ottawa_area_data.rename(columns={'Dim: Sex (3): Member ID: [1]: Total - Sex':'area_sq_km'})

In [ ]:
ottawa_census_gpd = ottawa_census_gpd.merge(right=ottawa_area_data, how='left',
                              left_on='CTUID', right_on='GEO_CODE (POR)')

#### Exploring difference in areas

In [ ]:
ottawa_census_gpd[['area', 'area_sq_km']].head()

In [ ]:
ottawa_census_gpd['area'] = ottawa_census_gpd['area']*10000

In [ ]:
ottawa_census_gpd['area_sq_km'] = pd.to_numeric(ottawa_census_gpd['area_sq_km'])
ottawa_census_gpd['area_dif'] = ottawa_census_gpd['area'] - ottawa_census_gpd['area_sq_km']

In [ ]:
# difference is always positive meaning that the value from geopandas is always larger.
# makes sense give that the census area should be the same but minus water
ottawa_census_gpd['area_dif'].describe()

In [ ]:
# Surprisingly high variance. Thought the lowest would be < 0.05
# Should follow up here more.
ottawa_census_gpd['area_dif_perc'] = ottawa_census_gpd['area_dif'] / ottawa_census_gpd['area_sq_km']
ottawa_census_gpd['area_dif_perc'].describe()

In [ ]:
ottawa_census_gpd.area_dif_perc.plot(kind='hist', bins=15, figsize=(13,5))

#### Mapping difference in areas

In [ ]:
# what is this one crazy tract?
# Answer: It's a tract along the river which is indeed mostly water
test = ottawa_census_gpd.copy(deep=True)
test[['geometry', 'area_dif_perc']].explore(column='area_dif_perc')

In [ ]:
# let's highlight the close ones
# Obervation: some really seem to have no water. Maybe it strips out parkland?
# Tried looking at CTUID == 5050038 which is nicely rectangular here:
# https://www.calcmaps.com/map-area/
# gives an area of 0.433115 km^2. This is much closer to the area_sq_km (0.43 vs 0.498)
# This implies there's something off with how I'm projecting this or with geopandas .area property
test = ottawa_census_gpd.copy(deep=True)
test.loc[ottawa_census_gpd.area_dif_perc<0.16, 'pop'] = -99999
test[['geometry', 'pop', 'area_dif_perc', 'area', 'area_sq_km', 'CTUID']].explore(column='pop')

### Determine which library is closest to a tract
Will use the centroids for this I think

In [ ]:
ottawa_census_display.centroid